In [25]:
%pip install altair

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from vcc_easy import *

block = os.urandom(VCC.IV_SIZE)
print(block)
# block = b'\x98\xbfJL\xdb\x90\x18\xa4\x0e\x83?\xe3\xdc\x1aPQ'
block = b'\xfa\x6e\xfa\xed\xaf\x83\xd2\xb8\xdf\xe1\xe4\xbb\xce\xe5\x10\x8c'
print(block)

def bytes_to_bits_str(b):
    return ''.join(format(x, '08b') for x in b)

def int_to_bits_str(i):
    # use bytes_to_bits_str
    return bytes_to_bits_str(i.to_bytes(16, byteorder='big'))

print(bytes_to_bits_str(block))
print(len(bytes_to_bits_str(block)))

b'\x1a3T\x9c\x1a8\x0b\x84\xd6Y\xf9\xefj\x12\xb9S'
b'\xfan\xfa\xed\xaf\x83\xd2\xb8\xdf\xe1\xe4\xbb\xce\xe5\x10\x8c'
11111010011011101111101011101101101011111000001111010010101110001101111111100001111001001011101111001110111001010001000010001100
128


In [27]:
from vcc_easy import *

class VCC_WithDebug(VCC):
    def __init__(self, key):
        super().__init__(key)
        
    def encrypt_debug(self, plaintext):
        plain_state = bytes2matrix(plaintext)        
        block = bytes_to_bits_str(plaintext)
        changes = [0] * len(block)
        add_round_key(plain_state, self._key_matrices[0])

        for i in range(1, self.n_rounds):
            s1_func, s2_func, s3_func = self.get_sbox_functions(i, False)
            
            s1_func(plain_state)
            shift_rows(plain_state)
            s2_func(plain_state)
            mix_columns(plain_state)
            s3_func(plain_state)
            add_round_key(plain_state, self._key_matrices[i])
            
            new_block = bytes_to_bits_str(matrix2bytes(plain_state))
            for j in range(len(block)):
                if block[j] != new_block[j]:
                    changes[j] += 1
            
            block = new_block

        return changes


In [28]:
vcc = VCC_WithDebug(b'\x00'*16)
vcc = VCC_WithDebug(b'\xaf3\x83z"j_\xa6\xc7*#\xdb\xdb\x89\xfaF')
block = b'\xfa\x6e\xfa\xed\xaf\x83\xd2\xb8\xdf\xe1\xe4\xbb\xce\xe5\x10\x8c'

changes = vcc.encrypt_debug(block)

In [29]:
def draw_cool_graph(changes):
    import altair as alt
    import pandas as pd

    x, y, z = [], [], []
    for i in range(len(changes)):
        x.append(i % 8)
        y.append(i // 8)
        z.append(changes[i])

    source = pd.DataFrame({'x': x, 'y': y, 'z': z})
    return alt.Chart(source).mark_rect().encode(
        x=alt.X('x:O', axis=alt.Axis(title='Bit')),
        y=alt.Y('y:O', axis=alt.Axis(title='Byte')),
        color=alt.Color('z:Q', title='Changes')
    ).properties(
        width=400,
        height=600
    )


draw_cool_graph(changes)

alt.Chart(...)

In [30]:
def auto_corr_test(data):
    # print("Автокорреляционный тест")
    bits = int.from_bytes(data, byteorder='big')
    entry_bytes = [1 if bits & (1 << k) else -1 for k in range(127, -1, -1)]

    D_list = []
    X_list = []
    for D in range(1, 65):
        A = 0
        for i in range(128 - D):
            A += entry_bytes[i] * entry_bytes[i + D]

        X = A / (128 - D)
        # print(f"для D = {D}: {X}")
        D_list.append(D)
        X_list.append(X)
    
    from rich.console import Console
    from rich.table import Table

    table = Table(title="Автокорелляционный тест")

    table.add_column("D")
    table.add_column("X")
    table.add_column("D")
    table.add_column("X")
    for i in range(32):
        table.add_row(str(D_list[i]), str(X_list[i]), str(D_list[i+32]), str(X_list[i+32]))
    # table.add_column("D", justify="right", style="cyan", no_wrap=True)
    # table.add_column("Title", style="magenta")
    # table.add_column("Box Office", justify="right", style="green")

    console = Console()
    console.print(table)

    import altair as alt
    import pandas as pd

    source = pd.DataFrame({
        'x': D_list,
        'y': X_list
    })

    return alt.Chart(source).mark_bar().encode(
        x='x',
        y='y'
    ).properties(
        width=400,
        height=400
    )


def series_test(data):
    print("Последовательностей разрывов")
    bits = int.from_bytes(data, byteorder='big')
    entry_nul = ""
    nul = ""
    one = ""

    for k in range(127, -1, -1):
        nul += "0"
        one += "1"
        entry_nul += "1" if bits & (1 << k) else "0"

    entry_one = entry_nul
    X = 0

    for i in range(127, -1, -1):
        count = entry_nul.count(nul)
        entry_nul = entry_nul.replace(nul, "")
        nul = nul[:-1]

        if count != 0:
            print(f"длиной {i+1} = {count}")
            e = (64 - i + 4) / (2 ** (i + 3))
            X += (count - e) ** 2 / e

    print()
    print("Последовательностей блоков")
    for i in range(127, -1, -1):
        count = entry_one.count(one)
        entry_one = entry_one.replace(one, "")
        one = one[:-1]

        if count != 0:
            print(f"длиной {i+1} = {count}")
            e = (127 - i + 4) / (2 ** (i + 4))
            X += (count - e) ** 2 / e

    print()
    print(f"Статистика Х = {X}")


def frequency_test(data):
    test = data
    sum0 = 0
    sum1 = 0
    bites = int.from_bytes(test, byteorder='big')

    for k in range(127, -1, -1):
        if bites & (1 << k):
            sum1 += 1
        else:
            sum0 += 1

    res0 = sum0 / 127
    res1 = sum1 / 127

    print(f"Частотный тест 0: {res0}")
    print(f"Частотный тест 1: {res1}")


def all_tests(data):
    series_test(data)
    print()
    frequency_test(data)
    print()
    return auto_corr_test(data)

encrypted = vcc.encrypt_block(block)

all_tests(encrypted)

Последовательностей разрывов
длиной 5 = 2
длиной 4 = 3
длиной 3 = 3
длиной 2 = 6
длиной 1 = 15

Последовательностей блоков
длиной 7 = 1
длиной 5 = 1
длиной 4 = 3
длиной 3 = 5
длиной 2 = 11
длиной 1 = 9

Статистика Х = 41.7307860106397

Частотный тест 0: 0.4566929133858268
Частотный тест 1: 0.5511811023622047



                  Автокорелляционный тест                  
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ D  ┃ X                     ┃ D  ┃ X                     ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1  │ 0.08661417322834646   │ 33 │ 0.07368421052631578   │
│ 2  │ -0.09523809523809523  │ 34 │ -0.0425531914893617   │
│ 3  │ -0.008                │ 35 │ -0.010752688172043012 │
│ 4  │ -0.11290322580645161  │ 36 │ 0.021739130434782608  │
│ 5  │ 0.04065040650406504   │ 37 │ 0.07692307692307693   │
│ 6  │ 0.01639344262295082   │ 38 │ -0.13333333333333333  │
│ 7  │ 0.0743801652892562    │ 39 │ -0.056179775280898875 │
│ 8  │ 0.05                  │ 40 │ 0.06818181818181818   │
│ 9  │ 0.025210084033613446  │ 41 │ 0.05747126436781609   │
│ 10 │ 0.03389830508474576   │ 42 │ 0.023255813953488372  │
│ 11 │ 0.09401709401709402   │ 43 │ -0.10588235294117647  │
│ 12 │ 0.0                   │ 44 │ -0.09523809523809523  │
│ 13 │ 0.008695652173913044  │ 45 │ -0.012048192771084338 │
│ 14 │ 0.14035087719298245   │ 46 │ -0.07317073170731707  │
│ 15 │ -0.07964601769911504  │ 47 │ -0.08641975308641975  │
│ 16 │ -0.08928571428571429  │ 48 │ 0.0                   │
│ 17 │ -0.06306306306306306  │ 49 │ -0.0379746835443038   │
│ 18 │ 0.12727272727272726   │ 50 │ 0.05128205128205128   │
│ 19 │ 0.10091743119266056   │ 51 │ 0.11688311688311688   │
│ 20 │ -0.09259259259259259  │ 52 │ -0.07894736842105263  │
│ 21 │ -0.04672897196261682  │ 53 │ 0.06666666666666667   │
│ 22 │ -0.09433962264150944  │ 54 │ 0.0                   │
│ 23 │ 0.12380952380952381   │ 55 │ -0.0958904109589041   │
│ 24 │ -0.07692307692307693  │ 56 │ 0.1111111111111111    │
│ 25 │ -0.08737864077669903  │ 57 │ -0.2112676056338028   │
│ 26 │ 0.0392156862745098    │ 58 │ -0.08571428571428572  │
│ 27 │ -0.0297029702970297   │ 59 │ 0.18840579710144928   │
│ 28 │ 0.02                  │ 60 │ 0.058823529411764705  │
│ 29 │ -0.050505050505050504 │ 61 │ -0.19402985074626866  │
│ 30 │ 0.04081632653061224   │ 62 │ -0.15151515151515152  │
│ 31 │ -0.05154639175257732  │ 63 │ -0.015384615384615385 │
│ 32 │ -0.125                │ 64 │ -0.0625               │
└────┴───────────────────────┴────┴───────────────────────┘

alt.Chart(...)